Установите [evxcr jupyter kernel](https://github.com/evcxr/evcxr/blob/main/evcxr_jupyter/README.md) или в colab активируйте данный скрипт

In [ ]:
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

In [51]:
:dep polars = { version = "x", features = ["lazy", "pivot"]}
:dep glob

In [52]:
use glob::glob;
use polars::{prelude::*, datatypes::PlSmallStr};
 
std::env::set_var("POLARS_FMT_TABLE_ROUNDED_CORNERS", "1"); // apply rounded corners to UTF8-styled tables.
std::env::set_var("POLARS_FMT_MAX_COLS", "20"); // maximum number of columns shown when formatting DataFrames.
std::env::set_var("POLARS_FMT_MAX_ROWS", "50"); // maximum number of rows shown when formatting DataFrames.
std::env::set_var("POLARS_FMT_STR_LEN", "50");  // maximum number of characters printed per string value.

#### Описание данных

В папке Data находится информация о студентах. Всего 10 групп студентов. Файлы делятся на две категории:
    * Students_info_i - информация о студентах из группы i
    * Students_marks_i - оценки студентов из группы i за экзамены

#### Задание 1. 

Соберите всю информацию о студентах в одну таблицу df. В получившейся таблице должна быть информация и оценки всех студентов из всех групп. Напечатайте несколько строк таблицы для демонстрации результата.¶

In [53]:
let infos_csv = "../../HW01/Data/Students_info_*.csv";
let mut infos_vec = vec![];

for path in glob(infos_csv).unwrap() {
    if let Ok(csv) = path {
        
        let lf = LazyCsvReader::new(csv)
            .finish()
            .unwrap();
        infos_vec.push(lf);
    }
}
    
let marks_csv = "../../HW01/Data/Students_marks_*.csv";
let mut marks_vec = vec![];

for path in glob(marks_csv).unwrap() {
    if let Ok(csv) = path {
        let lf = LazyCsvReader::new(csv)
            .finish()
            .unwrap();
        marks_vec.push(lf);
    }
}

let df_info = infos_vec.into_iter()
            .reduce(
                |acc, lf| concat(
                    [acc.clone(), lf.clone()], 
                    UnionArgs::default()
                ).unwrap())
            .unwrap();

let df_marks = marks_vec.into_iter()
            .reduce(
                |acc, lf| concat(
                    [acc.clone(), lf.clone()], 
                    UnionArgs::default()
                ).unwrap())
            .unwrap();

let df = df_info.join(
                    df_marks, 
                    [col("index")],
                    [col("index")],
                    JoinArgs::default(),
                )
                .collect()
                .unwrap();
df

shape: (1_000, 10)
╭───────┬────────┬──────────┬──────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────╮
│ index ┆ gender ┆ race/eth ┆ parental ┆ lunch   ┆ test    ┆ group   ┆ math    ┆ reading ┆ writing │
│ ---   ┆ ---    ┆ nicity   ┆ level of ┆ ---     ┆ prepara ┆ ---     ┆ score   ┆ score   ┆ score   │
│ i64   ┆ str    ┆ ---      ┆ educatio ┆ str     ┆ tion    ┆ str     ┆ ---     ┆ ---     ┆ ---     │
│       ┆        ┆ str      ┆ n        ┆         ┆ course  ┆         ┆ i64     ┆ i64     ┆ i64     │
│       ┆        ┆          ┆ ---      ┆         ┆ ---     ┆         ┆         ┆         ┆         │
│       ┆        ┆          ┆ str      ┆         ┆ str     ┆         ┆         ┆         ┆         │
╞═══════╪════════╪══════════╪══════════╪═════════╪═════════╪═════════╪═════════╪═════════╪═════════╡
│ 0     ┆ female ┆ group B  ┆ bachelor ┆ standar ┆ none    ┆ group1  ┆ 72      ┆ 72      ┆ 74      │
│       ┆        ┆          ┆ 's       ┆ d       ┆         ┆         ┆  

#### Задание 2. 

Удалите столбец index у полученной таблицы. Напечатайте первые 10 строк таблицы. Выведите на экран размеры полученной таблицы.

In [54]:
// если не работает, перезапустите предыдущую ячейку
let df = df.drop("index").unwrap();
df

shape: (1_000, 9)
╭────────┬───────────┬───────────┬───────────┬───────────┬─────────┬───────┬───────────┬───────────╮
│ gender ┆ race/ethn ┆ parental  ┆ lunch     ┆ test prep ┆ group   ┆ math  ┆ reading   ┆ writing   │
│ ---    ┆ icity     ┆ level of  ┆ ---       ┆ aration   ┆ ---     ┆ score ┆ score     ┆ score     │
│ str    ┆ ---       ┆ education ┆ str       ┆ course    ┆ str     ┆ ---   ┆ ---       ┆ ---       │
│        ┆ str       ┆ ---       ┆           ┆ ---       ┆         ┆ i64   ┆ i64       ┆ i64       │
│        ┆           ┆ str       ┆           ┆ str       ┆         ┆       ┆           ┆           │
╞════════╪═══════════╪═══════════╪═══════════╪═══════════╪═════════╪═══════╪═══════════╪═══════════╡
│ female ┆ group B   ┆ bachelor' ┆ standard  ┆ none      ┆ group1  ┆ 72    ┆ 72        ┆ 74        │
│        ┆           ┆ s degree  ┆           ┆           ┆         ┆       ┆           ┆           │
│ female ┆ group C   ┆ some      ┆ standard  ┆ completed ┆ group1  ┆ 69  

#### Задание 3. 

Выведите на экран статистические характеристики числовых столбцов таблицы (минимум, максимум, среднее значение, стандартное отклонение)

In [55]:
// они убрали этот метод в расте
// df.describe(None).unwrap();

#### Задание 4. 

Проверьте, есть ли в таблице пропущенные значения

In [56]:
df.clone()
    .lazy()
    .select([col(PlSmallStr::from_static("*")).is_null().any(false)])
    .collect()
    .unwrap()

shape: (1, 9)
╭────────┬────────────────┬───────────┬───────┬─────────────┬───────┬───────┬─────────┬─────────╮
│ gender ┆ race/ethnicity ┆ parental  ┆ lunch ┆ test        ┆ group ┆ math  ┆ reading ┆ writing │
│ ---    ┆ ---            ┆ level of  ┆ ---   ┆ preparation ┆ ---   ┆ score ┆ score   ┆ score   │
│ bool   ┆ bool           ┆ education ┆ bool  ┆ course      ┆ bool  ┆ ---   ┆ ---     ┆ ---     │
│        ┆                ┆ ---       ┆       ┆ ---         ┆       ┆ bool  ┆ bool    ┆ bool    │
│        ┆                ┆ bool      ┆       ┆ bool        ┆       ┆       ┆         ┆         │
╞════════╪════════════════╪═══════════╪═══════╪═════════════╪═══════╪═══════╪═════════╪═════════╡
│ false  ┆ false          ┆ false     ┆ false ┆ false       ┆ false ┆ false ┆ false   ┆ false   │
╰────────┴────────────────┴───────────┴───────┴─────────────┴───────┴───────┴─────────┴─────────╯

#### Задание 5. 

Выведите на экран средние баллы студентов по каждому предмету (math, reading, writing)

In [57]:
let math_mean = df.clone()
                .lazy()
                .select([col("math score").mean().alias("math mean"),
                        col("reading score").mean().alias("reading mean"),
                        col("writing score").mean().alias("writing mean"),])
                .collect()
                .unwrap();
math_mean


shape: (1, 3)
╭───────────┬──────────────┬──────────────╮
│ math mean ┆ reading mean ┆ writing mean │
│ ---       ┆ ---          ┆ ---          │
│ f64       ┆ f64          ┆ f64          │
╞═══════════╪══════════════╪══════════════╡
│ 66.089    ┆ 69.169       ┆ 68.054       │
╰───────────┴──────────────┴──────────────╯

#### Задание 6. 

Как зависят оценки от того, проходил ли студент курс для подготовки к сдаче экзамена (test preparation course)? Выведите на экран для каждого предмета в отдельности средний балл студентов, проходивших курс для подготовки к экзамену и не проходивших курс.

In [58]:
let no_prep = df.clone()
                .lazy()
                .filter(col("test preparation course").eq(lit("none")))
                .select([col("math score").mean().alias("math mean"),
                        col("reading score").mean().alias("reading mean"),
                        col("writing score").mean().alias("writing mean"),])
                .collect()
                .unwrap();

let prep = df.clone()
                .lazy()
                .filter(col("test preparation course").eq(lit("completed")))
                .select([col("math score").mean().alias("math mean"),
                        col("reading score").mean().alias("reading mean"),
                        col("writing score").mean().alias("writing mean"),])
                .collect()
                .unwrap();

println!("No prep: {no_prep:?}\n");
println!("Prep: {prep:?}");

No prep: shape: (1, 3)
╭───────────┬──────────────┬──────────────╮
│ math mean ┆ reading mean ┆ writing mean │
│ ---       ┆ ---          ┆ ---          │
│ f64       ┆ f64          ┆ f64          │
╞═══════════╪══════════════╪══════════════╡
│ 64.077882 ┆ 66.534268    ┆ 64.504673    │
╰───────────┴──────────────┴──────────────╯

Prep: shape: (1, 3)
╭───────────┬──────────────┬──────────────╮
│ math mean ┆ reading mean ┆ writing mean │
│ ---       ┆ ---          ┆ ---          │
│ f64       ┆ f64          ┆ f64          │
╞═══════════╪══════════════╪══════════════╡
│ 69.695531 ┆ 73.893855    ┆ 74.418994    │
╰───────────┴──────────────┴──────────────╯


#### Задание 7. 

Выведите на экран все различные значения из столбца lunch.

In [59]:
let res = df.clone()
            .lazy()
            .select([col("lunch").unique()])
            .collect()
            .unwrap();

res

shape: (2, 1)
╭──────────────╮
│ lunch        │
│ ---          │
│ str          │
╞══════════════╡
│ standard     │
│ free/reduced │
╰──────────────╯

#### Задание 8. 

Переименуйте колонку "parental level of education" в "education", а "test preparation course" в "test preparation" с помощью метода pandas rename
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html

In [60]:
let df = df.lazy()
        .rename(["parental level of education", "test preparation course"], ["education", "test preparation"], true)
        .collect()
        .unwrap();
df

shape: (1_000, 9)
╭────────┬───────────┬───────────┬───────────┬───────────┬─────────┬───────┬───────────┬───────────╮
│ gender ┆ race/ethn ┆ education ┆ lunch     ┆ test prep ┆ group   ┆ math  ┆ reading   ┆ writing   │
│ ---    ┆ icity     ┆ ---       ┆ ---       ┆ aration   ┆ ---     ┆ score ┆ score     ┆ score     │
│ str    ┆ ---       ┆ str       ┆ str       ┆ ---       ┆ str     ┆ ---   ┆ ---       ┆ ---       │
│        ┆ str       ┆           ┆           ┆ str       ┆         ┆ i64   ┆ i64       ┆ i64       │
╞════════╪═══════════╪═══════════╪═══════════╪═══════════╪═════════╪═══════╪═══════════╪═══════════╡
│ female ┆ group B   ┆ bachelor' ┆ standard  ┆ none      ┆ group1  ┆ 72    ┆ 72        ┆ 74        │
│        ┆           ┆ s degree  ┆           ┆           ┆         ┆       ┆           ┆           │
│ female ┆ group C   ┆ some      ┆ standard  ┆ completed ┆ group1  ┆ 69    ┆ 90        ┆ 88        │
│        ┆           ┆ college   ┆           ┆           ┆         ┆     

**Зафиксируем минимальный балл для сдачи экзамена**

In [61]:
let passmark = 50;

#### Задание 9. 

Ответьте на вопросы:

    * Какая доля студентов сдала экзамен по математике (passmark > 50)?
    * Какая доля студентов, проходивших курс подготовки к экзамену, сдала экзамен по математике?
    * Какая доля женщин, не проходивших курс подготовки к экзамену, не сдала экзамен по математике? 

In [62]:
// буду считать, что доля от всех студентов
let all = df.clone()
            .height();

let passed = 100. * df.clone()
                    .lazy()
                    .filter(col("math score")
                            .gt(passmark)
                        )
                    .collect()
                    .unwrap()
                    .height() as f64
                    / all as f64;
            
println!("Passed: {passed}%");


let passed_and_prep = 100. * df.clone()
                            .lazy()
                            .filter(col("math score")
                                    .gt(passmark)
                                .and(
                                    col("test preparation")
                                    .eq(lit("completed"))
                                )
                                )
                            .collect()
                            .unwrap()
                            .height() as f64
                            / all as f64;

println!("Passed: {passed_and_prep}%");


let women_no_prep = 100. * df.clone()
                            .lazy()
                            .filter(col("math score")
                                    .lt(passmark)
                                .and(
                                    col("test preparation")
                                    .eq(lit("none"))
                                ).and(
                                    col("gender")
                                    .eq(lit("female"))
                                    )
                                )
                            .collect()
                            .unwrap()
                            .height() as f64
                            / all as f64;

println!("Passed: {women_no_prep}%");



Passed: 85%
Passed: 32.5%
Passed: 7%


#### Задание 10. 

С помощью groupby выполните задания ниже. Также выведите время выполнения каждого из заданий.

    * Для каждой этнической группы выведите средний балл за экзамен по чтению
    * Для каждого уровня образования выведите минимальный балл за экзамен по письму

In [63]:
let reading = df.clone()
    .lazy()
    .group_by([col("race/ethnicity")])
    .agg([
        col("reading score").mean()
    ])
    .collect()
    .unwrap();
println!("Reading: {reading}");

let writing = df.clone()
    .lazy()
    .group_by([col("education")])
    .agg([
        col("writing score").min()
    ])
    .collect()
    .unwrap();
println!("Writing: {writing}");

Reading: shape: (5, 2)
╭────────────────┬───────────────╮
│ race/ethnicity ┆ reading score │
│ ---            ┆ ---           │
│ str            ┆ f64           │
╞════════════════╪═══════════════╡
│ group D        ┆ 70.030534     │
│ group B        ┆ 67.352632     │
│ group C        ┆ 69.103448     │
│ group E        ┆ 73.028571     │
│ group A        ┆ 64.674157     │
╰────────────────┴───────────────╯
Writing: shape: (6, 2)
╭────────────────────┬───────────────╮
│ education          ┆ writing score │
│ ---                ┆ ---           │
│ str                ┆ i64           │
╞════════════════════╪═══════════════╡
│ master's degree    ┆ 46            │
│ high school        ┆ 15            │
│ some college       ┆ 19            │
│ associate's degree ┆ 35            │
│ some high school   ┆ 10            │
│ bachelor's degree  ┆ 38            │
╰────────────────────┴───────────────╯


#### Задание 11.

Выполните задание 10 с помощью циклов. Сравните время выполнения.

In [64]:
// не хочу, я верю, что методами быстрее

#### Задание 12. 

Выведите на экран средние баллы студентов по каждому предмету в зависимости от пола и уровня образования. То есть должно получиться количество групп, равных 2 * (число уровней образования), и для каждой такой группы выыведите средний балл по каждому из предметов.

Это можно сделать с помощью сводных таблиц (pivot_table). Дополнительно о них почитать можно тут:

https://www.kaggle.com/kamilpolak/tutorial-how-to-use-pivot-table-in-pandas

In [65]:
pivot::pivot(&df.clone(), ["gender"], 
                    Some(["education"]), 
                    Some(["math score", "reading score", "writing score"]), 
                    true, 
                    Some(Expr::Agg(AggExpr::Mean(Arc::new(Expr::Column(PlSmallStr::from_static("*")))))),
                    None)
                    .unwrap()


unstable pivot not yet supported, using stable pivot


shape: (6, 7)
╭──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────╮
│ education    ┆ math score_ ┆ math        ┆ reading sco ┆ reading     ┆ writing sco ┆ writing     │
│ ---          ┆ female      ┆ score_male  ┆ re_female   ┆ score_male  ┆ re_female   ┆ score_male  │
│ str          ┆ ---         ┆ ---         ┆ ---         ┆ ---         ┆ ---         ┆ ---         │
│              ┆ f64         ┆ f64         ┆ f64         ┆ f64         ┆ f64         ┆ f64         │
╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ bachelor's   ┆ 68.349206   ┆ 70.581818   ┆ 77.285714   ┆ 68.090909   ┆ 78.380952   ┆ 67.654545   │
│ degree       ┆             ┆             ┆             ┆             ┆             ┆             │
│ some college ┆ 65.40678    ┆ 69.009259   ┆ 73.550847   ┆ 64.990741   ┆ 74.050847   ┆ 63.148148   │
│ master's     ┆ 66.5        ┆ 74.826087   ┆ 76.805556   ┆ 73.130435   ┆ 77.6

#### Задание 13. 

Сколько студентов успешно сдали экзамен по математике?

Создайте новый столбец в таблице df под названием Math_PassStatus и запишите в него F, если студент не сдал экзамен по математике (балл за экзамен < passmark), и P иначе.

Посчитайте количество студентов, сдавших и не сдавших экзамен по математике.

Сделайте аналогичные шаги для экзаменов по чтению и письму.

In [66]:
let df = df.lazy()
            .with_columns([
                when(col("math score").lt(passmark))
                .then(lit("F"))
                .otherwise(lit("P"))
                .alias("Math_PassStatus"),

                when(col("reading score").lt(passmark))
                .then(lit("F"))
                .otherwise(lit("P"))
                .alias("Reading_PassStatus"),

                when(col("writing score").lt(passmark))
                .then(lit("F"))
                .otherwise(lit("P"))
                .alias("Writing_PassStatus"),
            ])
            .collect()
            .unwrap();
df

shape: (1_000, 12)
╭────────┬────────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────╮
│ gender ┆ race/e ┆ educat ┆ lunch ┆ test  ┆ group ┆ math  ┆ readi ┆ writi ┆ Math_ ┆ Readi ┆ Writi │
│ ---    ┆ thnici ┆ ion    ┆ ---   ┆ prepa ┆ ---   ┆ score ┆ ng    ┆ ng    ┆ PassS ┆ ng_Pa ┆ ng_Pa │
│ str    ┆ ty     ┆ ---    ┆ str   ┆ ratio ┆ str   ┆ ---   ┆ score ┆ score ┆ tatus ┆ ssSta ┆ ssSta │
│        ┆ ---    ┆ str    ┆       ┆ n     ┆       ┆ i64   ┆ ---   ┆ ---   ┆ ---   ┆ tus   ┆ tus   │
│        ┆ str    ┆        ┆       ┆ ---   ┆       ┆       ┆ i64   ┆ i64   ┆ str   ┆ ---   ┆ ---   │
│        ┆        ┆        ┆       ┆ str   ┆       ┆       ┆       ┆       ┆       ┆ str   ┆ str   │
╞════════╪════════╪════════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╡
│ female ┆ group  ┆ bachel ┆ stand ┆ none  ┆ group ┆ 72    ┆ 72    ┆ 74    ┆ P     ┆ P     ┆ P     │
│        ┆ B      ┆ or's   ┆ ard   ┆       ┆ 1     ┆       ┆       ┆    

#### Задание 14. 

Сколько студентов успешно сдали все экзамены?

Создайте столбец OverAll_PassStatus и запишите в него для каждого студента 'F', если студент не сдал хотя бы один из трех экзаменов, а иначе 'P'.

Посчитайте количество студентов, которые сдали все экзамены.

In [67]:
let df = df.lazy()
            .with_column(
                when(
                    col("Math_PassStatus").eq(lit("F"))
                    .or(col("Reading_PassStatus").eq(lit("F")))
                    .or(col("Writing_PassStatus").eq(lit("F")))
                )
                .then(lit("F"))
                .otherwise(lit("P"))
                .alias("Overall_PassStatus"),
            )
            .collect()
            .unwrap();

let n = Series::new("".into(), &[10]);
df.sample_n(&n,false,true,None)

Ok(shape: (10, 13)
╭─────┬─────┬─────┬────────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────╮
│ gen ┆ rac ┆ edu ┆ lunch  ┆ test  ┆ group ┆ math  ┆ readi ┆ writi ┆ Math_ ┆ Readi ┆ Writi ┆ Overa │
│ der ┆ e/e ┆ cat ┆ ---    ┆ prepa ┆ ---   ┆ score ┆ ng    ┆ ng    ┆ PassS ┆ ng_Pa ┆ ng_Pa ┆ ll_Pa │
│ --- ┆ thn ┆ ion ┆ str    ┆ ratio ┆ str   ┆ ---   ┆ score ┆ score ┆ tatus ┆ ssSta ┆ ssSta ┆ ssSta │
│ str ┆ ici ┆ --- ┆        ┆ n     ┆       ┆ i64   ┆ ---   ┆ ---   ┆ ---   ┆ tus   ┆ tus   ┆ tus   │
│     ┆ ty  ┆ str ┆        ┆ ---   ┆       ┆       ┆ i64   ┆ i64   ┆ str   ┆ ---   ┆ ---   ┆ ---   │
│     ┆ --- ┆     ┆        ┆ str   ┆       ┆       ┆       ┆       ┆       ┆ str   ┆ str   ┆ str   │
│     ┆ str ┆     ┆        ┆       ┆       ┆       ┆       ┆       ┆       ┆       ┆       ┆       │
╞═════╪═════╪═════╪════════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╪═══════╡
│ fem ┆ gro ┆ ass ┆ free/r ┆ none  ┆ group ┆ 43    ┆ 60    ┆ 58    ┆ F  

#### Задание 15. 

Переведем баллы в оценки

*Система перевода баллов в оценки*

        [90;100] = A
        [80;90) = B
        [70;80) = C
        [60;70) = D
        [50;60) = E
        [0;50) = F (Fail)

Создайте вспомогательную функцию, которая будет по среднему баллу за три экзамена выставлять оценку студенту по данным выше критериям.

Создайте столбец Grade и запишите в него оценку каждого студента.

Выведите количество студентов, получивших каждую из оценок.

In [68]:
let res = df.clone()
            .lazy()
            .with_column(
                (col("math score") 
                + col("reading score") 
                + col("writing score"))
                .cast(DataType::Float64)
                .alias("average") 
                / lit(3.0)
            )
            .with_column(
                when(
                    col("average")
                    .lt(50)
                )
                .then(lit("F"))
                .when(
                    col("average")
                    .lt(60)
                    .alias("2")
                )
                .then(lit("E"))
                .when(
                    col("average")
                    .lt(70)
                )
                .then(lit("D"))
                .when(
                    col("average")
                    .lt(80)
                )
                .then(lit("C"))
                .when(
                    col("average")
                    .lt(90)
                )
                .then(lit("B"))
                .otherwise(lit("A"))
                .alias("Grade")
            )
            .drop([col("average")])
            .collect()
            .unwrap();
res

shape: (1_000, 14)
╭─────┬─────┬─────┬─────┬─────┬─────┬─────┬────────┬───────┬───────┬───────┬───────┬───────┬───────╮
│ gen ┆ rac ┆ edu ┆ lun ┆ tes ┆ gro ┆ mat ┆ readin ┆ writi ┆ Math_ ┆ Readi ┆ Writi ┆ Overa ┆ Grade │
│ der ┆ e/e ┆ cat ┆ ch  ┆ t   ┆ up  ┆ h   ┆ g      ┆ ng    ┆ PassS ┆ ng_Pa ┆ ng_Pa ┆ ll_Pa ┆ ---   │
│ --- ┆ thn ┆ ion ┆ --- ┆ pre ┆ --- ┆ sco ┆ score  ┆ score ┆ tatus ┆ ssSta ┆ ssSta ┆ ssSta ┆ str   │
│ str ┆ ici ┆ --- ┆ str ┆ par ┆ str ┆ re  ┆ ---    ┆ ---   ┆ ---   ┆ tus   ┆ tus   ┆ tus   ┆       │
│     ┆ ty  ┆ str ┆     ┆ ati ┆     ┆ --- ┆ i64    ┆ i64   ┆ str   ┆ ---   ┆ ---   ┆ ---   ┆       │
│     ┆ --- ┆     ┆     ┆ on  ┆     ┆ i64 ┆        ┆       ┆       ┆ str   ┆ str   ┆ str   ┆       │
│     ┆ str ┆     ┆     ┆ --- ┆     ┆     ┆        ┆       ┆       ┆       ┆       ┆       ┆       │
│     ┆     ┆     ┆     ┆ str ┆     ┆     ┆        ┆       ┆       ┆       ┆       ┆       ┆       │
╞═════╪═════╪═════╪═════╪═════╪═════╪═════╪════════╪═══════╪═══════╪════

In [ ]:
// если у нас больше 3 столбцов, то считать mean вручную тупо, поэтому можно так
// (украл у deepseek, так как документация polars rust просто ужасна)

let score_columns = df
        .clone()
        .schema()
        .iter_names_cloned()
        .filter(|name| name.ends_with("score"))
        .collect::<Vec<_>>();

let sum_expr = score_columns
        .iter()
        .fold(lit(0.0), |acc, col_name| acc + col(PlSmallStr::from_str(col_name)).cast(DataType::Float64));


let count = score_columns.len() as f64;
let average_expr = (sum_expr / lit(count)).alias("average");

let df = df.lazy()
        .with_column(average_expr.clone())
        .with_column(
            when(col("average").lt(50.0))
                .then(lit("F"))
                .when(col("average").lt(60.0))
                .then(lit("E"))
                .when(col("average").lt(70.0))
                .then(lit("D"))
                .when(col("average").lt(80.0))
                .then(lit("C"))
                .when(col("average").lt(90.0))
                .then(lit("B"))
                .otherwise(lit("A"))
                .alias("Grade")
        )
        .drop(["average"])
        .collect()
        .unwrap();

df

shape: (1_000, 14)
╭─────┬─────┬─────┬─────┬─────┬─────┬─────┬────────┬───────┬───────┬───────┬───────┬───────┬───────╮
│ gen ┆ rac ┆ edu ┆ lun ┆ tes ┆ gro ┆ mat ┆ readin ┆ writi ┆ Math_ ┆ Readi ┆ Writi ┆ Overa ┆ Grade │
│ der ┆ e/e ┆ cat ┆ ch  ┆ t   ┆ up  ┆ h   ┆ g      ┆ ng    ┆ PassS ┆ ng_Pa ┆ ng_Pa ┆ ll_Pa ┆ ---   │
│ --- ┆ thn ┆ ion ┆ --- ┆ pre ┆ --- ┆ sco ┆ score  ┆ score ┆ tatus ┆ ssSta ┆ ssSta ┆ ssSta ┆ str   │
│ str ┆ ici ┆ --- ┆ str ┆ par ┆ str ┆ re  ┆ ---    ┆ ---   ┆ ---   ┆ tus   ┆ tus   ┆ tus   ┆       │
│     ┆ ty  ┆ str ┆     ┆ ati ┆     ┆ --- ┆ i64    ┆ i64   ┆ str   ┆ ---   ┆ ---   ┆ ---   ┆       │
│     ┆ --- ┆     ┆     ┆ on  ┆     ┆ i64 ┆        ┆       ┆       ┆ str   ┆ str   ┆ str   ┆       │
│     ┆ str ┆     ┆     ┆ --- ┆     ┆     ┆        ┆       ┆       ┆       ┆       ┆       ┆       │
│     ┆     ┆     ┆     ┆ str ┆     ┆     ┆        ┆       ┆       ┆       ┆       ┆       ┆       │
╞═════╪═════╪═════╪═════╪═════╪═════╪═════╪════════╪═══════╪═══════╪════